In [ ]:
import json, requests
import numpy as np
import yaml

### Import secret Keys to use with Foursquare API

In [ ]:
with open('/Users/ReddingSkinnyRobot/.secrets/foursquare_api.yaml') as f:
    secrets = yaml.load(f)

### Calculate bounding limits from http://boundingbox.klokantech.com/ TSV format

In [ ]:
westlimit=-122.459696 #Longitude
southlimit=47.491912  #Latitude
eastlimit=-122.224433 #Longitude
northlimit=47.734145  #Latitude

In [ ]:
km_east_west = (eastlimit - westlimit) * 97
km_north_south = (northlimit - southlimit) * 111

In [ ]:
km_east_west/100*1000

In [ ]:
km_east_west*100/100 #Meters in each grid east west

In [ ]:
km_north_south/100*1000 #meters in each grid north south

In [ ]:
number_of_grid_lines = 1000

In [ ]:
meters_per_ns_grid_line = km_north_south / number_of_grid_lines * 1000
meters_per_ew_grid_line = km_east_west / number_of_grid_lines * 1000

Difference between degrees of Longitude is about 60 miles (97km) 

Difference between degrees of Latitude is about 69 miles (111km)

### Generate longitude & latitude grids and define search functions

In [ ]:
number_of_gridlines = 100
longitude_grid = np.linspace(westlimit, eastlimit, number_of_gridlines)
latitude_grid = np.linspace(northlimit, southlimit, number_of_gridlines)

def populate_search_params(lat, long):
    search_params = dict(
        client_id=secrets['client_id'],
        client_secret=secrets['client_secret'],
        ll = '{}, {}'.format(lat, long),
        intent='browse',
        radius='200', #Meters
        limit='50',
        categoryId='4bf58dd8d48988d1e0931735',#Coffee shop
        llAcc='1',#Accuracy of lat & long in meters
        v='20180113' #Date of current version
        )
    return search_params

### Divide search into smaller groups to avoid breakage during API GET requests

In [ ]:
longitude_group1 = longitude_grid[:10]
longitude_group2 = longitude_grid[10:20]
longitude_group3 = longitude_grid[20:30]
longitude_group4 = longitude_grid[30:40]
longitude_group5 = longitude_grid[40:50]
longitude_group6 = longitude_grid[50:60]
longitude_group7 = longitude_grid[60:70]
longitude_group8 = longitude_grid[70:80]
longitude_group9 = longitude_grid[80:90]
longitude_group10 = longitude_grid[90:]

In [ ]:
longitude_groups = []

In [ ]:
longitude_groups.append(longitude_group1)
longitude_groups.append(longitude_group2)
longitude_groups.append(longitude_group3)
longitude_groups.append(longitude_group4)
longitude_groups.append(longitude_group5)
longitude_groups.append(longitude_group6)
longitude_groups.append(longitude_group7)
longitude_groups.append(longitude_group8)
longitude_groups.append(longitude_group9)
longitude_groups.append(longitude_group10)

### Search for all venues that match my search params

In [ ]:
search_url = 'https://api.foursquare.com/v2/venues/search'
venue_search_list = []

for group_i, longitude_group in enumerate(longitude_groups):
    current_group_search_list = []
    for long_i, long in enumerate(longitude_group):
        for lat in latitude_grid:
            search_params = populate_search_params(lat, long)
            try:
                search_resp = requests.get(url=search_url, params=search_params)
                search_data = json.loads(search_resp.text)
                current_group_search_list.append(search_data)
            except:
                current_group_search_list.append((lat, long))
    venue_search_list = venue_search_list + current_group_search_list
    print('Step {} of 10'.format(group_i+1))

In [ ]:
while True:
    try:
        break
        #open('fdfsf')
    except ZeroDivisionError:
        print("oops")
    except FileNotFoundError:
        print("didn't find file")
    finally:
        print('in finally')

In [ ]:
tips_params = dict(
    client_id=secrets['client_id'],
    client_secret=secrets['client_secret'],
    limit=500,
    v='20180113')

In [ ]:
tips_resp = requests.get(url=tips_url, params=tips_params)
tips_data = json.loads(tips_resp.text)

In [ ]:
tips_data

In [ ]:
tips = []
for tip in tips_data['response']['tips']['items']:
    tips.append(tip['text'])

In [ ]:
id_s = []
for venue in search_data['response']['venues']:
    id_s.append((venue['id'], venue['name'], venue['location']['address']))

In [ ]:
id_s

In [ ]:
def get_tips(venue_id):
    tip_list = []
    tips_url = 'https://api.foursquare.com/v2/venues/{}/tips'.format(venue_id)
    tips_params = dict(
    client_id=secrets['client_id'],
    client_secret=secrets['client_secret'],
    limit=500,
    v='20180113')
    tips_resp = requests.get(url=tips_url, params=tips_params)
    tips_data = json.loads(tips_resp.text)
    for tip in tips_data['response']['tips']['items']:
        tip_list.append(tip['text'])
    return tip_list

In [ ]:
all_tips = []
for venue_id, name, address in id_s:
    tip_list = get_tips(venue_id)
    all_tips.append((venue_id, tip_list))

In [ ]:
all_tips